In [ ]:
!pip install tqdm

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
from collections import Counter
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imputedt/test_data_imputed.csv
/kaggle/input/post-processing-done/processed.csv


In [2]:
df = pd.read_csv("/kaggle/input/post-processing-done/processed.csv")
df

,Unnamed: 0,ACCT_AGE,LIMIT,OUTS,ACCT_RESIDUAL_TENURE,LOAN_TENURE,INSTALAMT,SI_FLG,AGE,VINTAGE,...,INCOME_BAND1_G,INCOME_BAND1_H,AGREG_GROUP_#Housing Loan,AGREG_GROUP_#Total Auto Loan,AGREG_GROUP_#Total Xpress Credit,PRODUCT_TYPE_EDUCATION LOAN,PRODUCT_TYPE_HOME LOAN,PRODUCT_TYPE_PERSONAL LOAN,TIME_PERIOD_JAN25,TIME_PERIOD_NOV24
0,0,1.613,1005500.0,494161.89,0.890,914,38513.0,1,57.663,18.601,...,True,False,False,False,True,False,False,True,False,True
1,1,1.783,1005500.0,428072.24,0.720,914,38513.0,1,57.833,18.771,...,True,False,False,False,True,False,False,True,True,False
2,2,1.698,1005500.0,461364.10,0.805,914,38513.0,1,57.748,18.686,...,True,False,False,False,True,False,False,True,False,False
3,3,9.127,1005500.0,1204287.25,17.878,9862,12736.0,1,52.302,14.039,...,False,False,True,False,False,False,True,False,False,True
4,4,9.296,1005500.0,1203224.25,17.708,9862,12736.0,1,52.472,14.209,...,False,False,True,False,False,False,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325828,327736,11.837,185161.0,166393.79,4.411,5934,3688.0,1,61.483,17.399,...,False,False,False,False,False,True,False,False,False,True
325829,327737,2.516,735500.0,529552.88,3.486,2192,14310.0,1,27.482,8.902,...,False,False,False,False,True,False,False,True,True,False
325830,327738,2.944,705500.0,849950.35,13.059,5844,9770.0,0,25.286,7.062,...,False,False,False,False,False,True,False,False,False,True
325831,327739,1.021,205500.0,169529.00,1.980,1096,6865.0,1,34.353,13.261,...,False,False,False,False,True,False,False,True,True,False


In [3]:
test = pd.read_csv("/kaggle/input/imputedt/test_data_imputed.csv")
test.isnull().sum().sum()

0

In [4]:
test

,Unnamed: 0,ACCT_AGE,LIMIT,OUTS,ACCT_RESIDUAL_TENURE,LOAN_TENURE,INSTALAMT,SI_FLG,AGE,VINTAGE,...,INCOME_BAND1_F,INCOME_BAND1_G,INCOME_BAND1_H,AGREG_GROUP_#Housing Loan,AGREG_GROUP_#Total Auto Loan,AGREG_GROUP_#Total Xpress Credit,PRODUCT_TYPE_EDUCATION LOAN,PRODUCT_TYPE_HOME LOAN,PRODUCT_TYPE_PERSONAL LOAN,TIME_PERIOD_MAR25
0,0,2.694,729200.0,541543.71,3.308,2192,15247.0,1,38.915,18.765,...,False,False,False,False,False,True,False,False,True,False
1,1,5.652,980500.0,426219.82,2.349,2922,15836.0,1,51.436,15.665,...,True,False,False,False,True,False,False,False,False,False
2,2,5.737,980500.0,413595.82,2.265,2922,15836.0,1,51.521,15.750,...,True,False,False,False,True,False,False,False,False,True
3,3,6.479,735500.0,221620.79,1.607,2953,11996.0,1,33.526,14.702,...,True,False,False,False,True,False,False,False,False,True
4,4,6.394,735500.0,231762.79,1.692,2953,11996.0,1,33.441,14.617,...,True,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190676,191688,1.561,461500.0,407286.13,4.441,2192,9409.0,0,27.901,11.054,...,False,False,False,False,False,True,False,False,True,True
190677,191689,3.020,495500.0,309105.77,2.982,2192,9165.0,1,39.862,9.866,...,False,False,False,False,False,True,False,False,True,True
190678,191690,3.097,805500.0,545336.58,2.903,2191,16169.0,1,42.127,13.100,...,False,True,False,False,False,True,False,False,True,True
190679,191691,2.336,625500.0,459398.55,3.666,2192,11977.0,1,25.552,9.838,...,False,False,False,False,False,True,False,False,True,False


In [ ]:
# Lists of mismatched columns
cols_only_in_df = [
    'CREDIT_HISTORY_LENGTH1_28yrs 3mon', 'CREDIT_HISTORY_LENGTH1_40yrs 11mon', 'CREDIT_HISTORY_LENGTH1_28yrs 5mon',
    'AVERAGE_ACCT_AGE1_23yrs 3mon', 'CREDIT_HISTORY_LENGTH1_45yrs 7mon', 'CREDIT_HISTORY_LENGTH1_43yrs 9mon',
    'AVERAGE_ACCT_AGE1_18yrs 0mon', 'CREDIT_HISTORY_LENGTH1_28yrs 2mon', 'CREDIT_HISTORY_LENGTH1_25yrs 8mon',
    'AVERAGE_ACCT_AGE1_18yrs 1mon', 'CREDIT_HISTORY_LENGTH1_40yrs 1mon', 'AVERAGE_ACCT_AGE1_22yrs 5mon',
    'TIME_PERIOD_JAN25', 'AVERAGE_ACCT_AGE1_17yrs 7mon', 'CREDIT_HISTORY_LENGTH1_32yrs 0mon',
    'AVERAGE_ACCT_AGE1_19yrs 6mon', 'CREDIT_HISTORY_LENGTH1_30yrs 2mon', 'CREDIT_HISTORY_LENGTH1_46yrs 5mon',
    'CREDIT_HISTORY_LENGTH1_32yrs 9mon', 'CREDIT_HISTORY_LENGTH1_33yrs 5mon', 'AVERAGE_ACCT_AGE1_21yrs 0mon',
    'TIME_PERIOD_NOV24', 'CREDIT_HISTORY_LENGTH1_30yrs 9mon', 'AVERAGE_ACCT_AGE1_19yrs 0mon',
    'CREDIT_HISTORY_LENGTH1_39yrs 0mon', 'CREDIT_HISTORY_LENGTH1_29yrs 1mon', 'AVERAGE_ACCT_AGE1_21yrs 6mon',
    'CREDIT_HISTORY_LENGTH1_26yrs 4mon', 'AVERAGE_ACCT_AGE1_19yrs 4mon', 'CREDIT_HISTORY_LENGTH1_33yrs 11mon',
    'CREDIT_HISTORY_LENGTH1_41yrs 9mon', 'CREDIT_HISTORY_LENGTH1_36yrs 6mon', 'CREDIT_HISTORY_LENGTH1_28yrs 10mon',
    'CREDIT_HISTORY_LENGTH1_30yrs 8mon', 'CREDIT_HISTORY_LENGTH1_28yrs 9mon', 'CREDIT_HISTORY_LENGTH1_32yrs 5mon',
    'TARGET',  # <- We will preserve this
    'CREDIT_HISTORY_LENGTH1_26yrs 8mon', 'CREDIT_HISTORY_LENGTH1_36yrs 0mon', 'AVERAGE_ACCT_AGE1_23yrs 0mon',
    'AVERAGE_ACCT_AGE1_22yrs 9mon', 'CREDIT_HISTORY_LENGTH1_28yrs 1mon', 'CREDIT_HISTORY_LENGTH1_41yrs 7mon',
    'CREDIT_HISTORY_LENGTH1_30yrs 3mon', 'CREDIT_HISTORY_LENGTH1_26yrs 5mon', 'CREDIT_HISTORY_LENGTH1_37yrs 7mon',
    'CREDIT_HISTORY_LENGTH1_27yrs 11mon', 'AVERAGE_ACCT_AGE1_21yrs 7mon', 'AVERAGE_ACCT_AGE1_17yrs 9mon',
    'CREDIT_HISTORY_LENGTH1_35yrs 6mon', 'CREDIT_HISTORY_LENGTH1_47yrs 10mon', 'AVERAGE_ACCT_AGE1_18yrs 7mon',
    'CREDIT_HISTORY_LENGTH1_39yrs 5mon', 'CREDIT_HISTORY_LENGTH1_28yrs 0mon'
]

cols_only_in_test = [
    'AVERAGE_ACCT_AGE1_20yrs 8mon', 'CREDIT_HISTORY_LENGTH1_40yrs 0mon', 'TIME_PERIOD_MAR25',
    'CREDIT_HISTORY_LENGTH1_29yrs 11mon', 'CREDIT_HISTORY_LENGTH1_27yrs 5mon', 'CREDIT_HISTORY_LENGTH1_35yrs 4mon',
    'CREDIT_HISTORY_LENGTH1_32yrs 3mon', 'CREDIT_HISTORY_LENGTH1_31yrs 5mon', 'KYC_FLG_1.1'
]

#Removing 'TARGET' from drop list
cols_only_in_df = [col for col in cols_only_in_df if col != 'TARGET']

# Dropping mismatched columns
df = df.drop(columns=cols_only_in_df, errors='ignore')
test = test.drop(columns=cols_only_in_test, errors='ignore')


common_cols = df.columns.intersection(test.columns)
print(f"df and test have {len(common_cols)} matching columns (excluding TARGET).")


df and test have 751 matching columns (excluding TARGET).


In [ ]:


# Separate features and target
X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Identify column types
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
boolean_cols = X.select_dtypes(include=['bool']).columns

# Preprocessing pipeline
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

boolean_transformer = Pipeline(steps=[
    ('passthrough', 'passthrough')  
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols),
        ('bool', boolean_transformer, boolean_cols)
    ])


X.shape

(325833, 751)

# DECISION TREE

In [ ]:
# Create pipeline with preprocessing and logistic regression
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(
        class_weight='balanced',  
        max_iter=100,
        random_state=42
    ))
])

# Data splitting into train and test data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Training the model
model.fit(X_train, y_train)

# Evaluation of trained model
train_pred = model.predict(X_train)
test_pred = model.predict(X_test)

print("\nTraining Report:")
print(classification_report(y_train, train_pred))

print("\nTest Report:")
print(classification_report(y_test, test_pred))

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



Training Report:
              precision    recall  f1-score   support

           0       0.96      0.74      0.83    232553
           1       0.26      0.76      0.39     28113

    accuracy                           0.74    260666
   macro avg       0.61      0.75      0.61    260666
weighted avg       0.89      0.74      0.79    260666


Test Report:
              precision    recall  f1-score   support

           0       0.96      0.74      0.83     58139
           1       0.26      0.76      0.38      7028

    accuracy                           0.74     65167
   macro avg       0.61      0.75      0.61     65167
weighted avg       0.89      0.74      0.78     65167



# CATBOOST

In [ ]:



X = df.drop('TARGET', axis=1)
y = df['TARGET']

# Identifying categorical feature indices
cat_features_indices = [i for i, col in enumerate(X.columns) if col in categorical_cols]

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Initializing CatBoost
cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    loss_function='Logloss',
    eval_metric='AUC',
    random_seed=42,
    scale_pos_weight=9,  
    cat_features=cat_features_indices,
    verbose=100  
)

# Train the model
cat_model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    early_stopping_rounds=50,
    plot=True  
)

# Evaluate
train_pred = cat_model.predict(X_train)
test_pred = cat_model.predict(X_test)

print("\nTraining Report:")
print(classification_report(y_train, train_pred))
print(f"Training ROC-AUC: {roc_auc_score(y_train, cat_model.predict_proba(X_train)[:,1])}")

print("\nTest Report:")
print(classification_report(y_test, test_pred))
print(f"Test ROC-AUC: {roc_auc_score(y_test, cat_model.predict_proba(X_test)[:,1])}")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	test: 0.8153122	best: 0.8153122 (0)	total: 281ms	remaining: 2m 20s
100:	test: 0.8847418	best: 0.8847418 (100)	total: 17.5s	remaining: 1m 9s
200:	test: 0.8967203	best: 0.8967203 (200)	total: 31.2s	remaining: 46.4s
300:	test: 0.9069054	best: 0.9069054 (300)	total: 44.2s	remaining: 29.2s
400:	test: 0.9122526	best: 0.9122526 (400)	total: 57.3s	remaining: 14.1s
499:	test: 0.9152941	best: 0.9152941 (499)	total: 1m 10s	remaining: 0us

bestTest = 0.915294113
bestIteration = 499


Training Report:
              precision    recall  f1-score   support

           0       0.98      0.82      0.89    232553
           1       0.37      0.88      0.52     28113

    accuracy                           0.83    260666
   macro avg       0.68      0.85      0.71    260666
weighted avg       0.92      0.83      0.85    260666

Training ROC-AUC: 0.9245268498465331

Test Report:
              precision    recall  f1-score   support

           0       0.98      0.82      0.89     58139
           1    

# XGBoost

In [ ]:
X = df.drop('TARGET', axis=1)
y = df['TARGET']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Converting categorical columns to int
for col in categorical_cols:
    X_train[col] = X_train[col].astype('category')
    X_test[col] = X_test[col].astype('category')


for col in boolean_cols:
    X_train[col] = X_train[col].astype(int)
    X_test[col] = X_test[col].astype(int)


xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    n_estimators=100,
    max_depth=3,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=9,  
    random_state=42,
    eval_metric='aucpr'  
)

xgb_model.fit(X_train, y_train)

# Evaluate
train_pred = xgb_model.predict(X_train)
test_pred = xgb_model.predict(X_test)

print("\nTraining Report:")
print(classification_report(y_train, train_pred))
print(f"Training ROC-AUC: {roc_auc_score(y_train, xgb_model.predict_proba(X_train)[:,1])}")

print("\nTest Report:")
print(classification_report(y_test, test_pred))
print(f"Test ROC-AUC: {roc_auc_score(y_test, xgb_model.predict_proba(X_test)[:,1])}")


Training Report:
              precision    recall  f1-score   support

           0       0.97      0.78      0.87    232553
           1       0.32      0.83      0.46     28113

    accuracy                           0.79    260666
   macro avg       0.65      0.81      0.66    260666
weighted avg       0.90      0.79      0.82    260666

Training ROC-AUC: 0.8887751039864978

Test Report:
              precision    recall  f1-score   support

           0       0.97      0.78      0.87     58139
           1       0.31      0.83      0.45      7028

    accuracy                           0.79     65167
   macro avg       0.64      0.80      0.66     65167
weighted avg       0.90      0.79      0.82     65167

Test ROC-AUC: 0.8861497896583153


# SMOTE


In [ ]:

X = df.drop('TARGET', axis=1)
y = df['TARGET']


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()
encoders = {}

# Encoding categorical variables
for col in categorical_cols:
    if col in X_train.columns:
        le = LabelEncoder()
        X_train_encoded[col] = le.fit_transform(X_train[col].astype(str))
        X_test_encoded[col] = le.transform(X_test[col].astype(str))
        encoders[col] = le

def manual_smote(X, y, target_class, k_neighbors=5, random_state=42):
    """
    Manual SMOTE implementation for generating synthetic samples
    """
    np.random.seed(random_state)
    
    # Get minority class samples and convert to numpy array
    minority_mask = y == target_class
    minority_samples = X[minority_mask].values.astype(float)
    
    if len(minority_samples) <= k_neighbors:
        k_neighbors = len(minority_samples) - 1
        
    if k_neighbors <= 0:
        print("Not enough minority samples for SMOTE")
        return np.array([])
    
    # Fit KNN on minority samples
    knn = NearestNeighbors(n_neighbors=k_neighbors + 1)
    knn.fit(minority_samples)
    
    synthetic_samples = []

    print(f"Generating synthetic samples for {len(minority_samples)} minority samples...")
    for i, sample in enumerate(tqdm(minority_samples, desc="SMOTE Progress", unit="sample")):
        distances, indices = knn.kneighbors([sample])
        neighbor_indices = indices[0][1:]
        
        for neighbor_idx in neighbor_indices:
            neighbor = minority_samples[neighbor_idx]
            gap = np.random.random()
            synthetic = sample + gap * (neighbor - sample)
            synthetic_samples.append(synthetic)
    
    return np.array(synthetic_samples)


print("Original class distribution:")
print(y_train.value_counts())


X_train_encoded = X_train.copy()
X_test_encoded = X_test.copy()
encoders = {}

for col in categorical_cols:
    if col in X_train.columns:
        le = LabelEncoder()
        X_train_encoded[col] = le.fit_transform(X_train[col].astype(str))
        X_test_encoded[col] = le.transform(X_test[col].astype(str))
        encoders[col] = le

# Get class counts
class_counts = Counter(y_train)
minority_class = min(class_counts, key=class_counts.get)
majority_class = max(class_counts, key=class_counts.get)

print(f"Minority class: {minority_class} (count: {class_counts[minority_class]})")
print(f"Majority class: {majority_class} (count: {class_counts[majority_class]})")


num_synthetic = class_counts[majority_class] - class_counts[minority_class]
print(f"Generating {num_synthetic} synthetic samples...")


synthetic_X = manual_smote(X_train_encoded, y_train, minority_class, k_neighbors=5)

# Randomly selecting required number of samples
if len(synthetic_X) > num_synthetic:
    indices = np.random.choice(len(synthetic_X), num_synthetic, replace=False)
    synthetic_X = synthetic_X[indices]


synthetic_y = np.full(len(synthetic_X), minority_class)


X_train_smote = np.vstack([X_train_encoded.values, synthetic_X])
y_train_smote = np.hstack([y_train.values, synthetic_y])

print(f"\nAfter manual SMOTE class distribution:")
print(f"Class {minority_class}: {np.sum(y_train_smote == minority_class)}")
print(f"Class {majority_class}: {np.sum(y_train_smote == majority_class)}")


X_train_smote = pd.DataFrame(X_train_smote, columns=X_train.columns)


X_train_smote_original = X_train_smote.copy()
for col in categorical_cols:
    if col in X_train_smote.columns:
        encoded_values = X_train_smote[col].round().astype(int)
        min_val = 0
        max_val = len(encoders[col].classes_) - 1
        encoded_values = np.clip(encoded_values, min_val, max_val)
        X_train_smote_original[col] = encoders[col].inverse_transform(encoded_values)


Original class distribution:
TARGET
0    232553
1     28113
Name: count, dtype: int64
Minority class: 1 (count: 28113)
Majority class: 0 (count: 232553)
Generating 204440 synthetic samples...
Generating synthetic samples for 28113 minority samples...


SMOTE Progress: 100%|██████████| 28113/28113 [11:02<00:00, 42.44sample/s]



After manual SMOTE class distribution:
Class 1: 168678
Class 0: 232553


# SMOTE + XGBoost

In [ ]:



def convert_to_numeric(df):
    df_numeric = df.copy()
    for col in df_numeric.columns:
        if df_numeric[col].dtype == 'object':
            
            df_numeric[col] = pd.to_numeric(df_numeric[col], errors='coerce')
           
            df_numeric[col] = df_numeric[col].fillna(0)
    return df_numeric

# Convert all datasets to numeric
X_train_smote_numeric = convert_to_numeric(pd.DataFrame(X_train_smote, columns=X_train.columns))
X_train_encoded_numeric = convert_to_numeric(X_train_encoded)
X_test_encoded_numeric = convert_to_numeric(X_test_encoded)

print("Data types after conversion:")
print(f"Training data types: {X_train_smote_numeric.dtypes.value_counts()}")
print(f"Test data types: {X_test_encoded_numeric.dtypes.value_counts()}")


xgb_model = xgb.XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='auc',
    early_stopping_rounds=50,
)


xgb_model.fit(
    X_train_smote_numeric, y_train_smote,
    eval_set=[(X_test_encoded_numeric, y_test)],
    verbose=100
)

# Evaluate trained model on original training and test data
train_pred = xgb_model.predict(X_train_encoded_numeric)
test_pred = xgb_model.predict(X_test_encoded_numeric)

print("\nXGBoost Training Report (Original Training Data):")
print(classification_report(y_train, train_pred))
print(f"Training ROC-AUC: {roc_auc_score(y_train, xgb_model.predict_proba(X_train_encoded_numeric)[:,1]):.4f}")

print("\nXGBoost Test Report:")
print(classification_report(y_test, test_pred))
print(f"Test ROC-AUC: {roc_auc_score(y_test, xgb_model.predict_proba(X_test_encoded_numeric)[:,1]):.4f}")

Data types after conversion:
Training data types: float64    751
Name: count, dtype: int64
Test data types: bool       610
float64    126
int64       15
Name: count, dtype: int64
[0]	validation_0-auc:0.80547
[100]	validation_0-auc:0.87513
[200]	validation_0-auc:0.89800
[300]	validation_0-auc:0.90866
[400]	validation_0-auc:0.91454
[499]	validation_0-auc:0.91746

XGBoost Training Report (Original Training Data):
              precision    recall  f1-score   support

           0       0.94      0.98      0.96    232553
           1       0.78      0.46      0.58     28113

    accuracy                           0.93    260666
   macro avg       0.86      0.72      0.77    260666
weighted avg       0.92      0.93      0.92    260666

Training ROC-AUC: 0.9424

XGBoost Test Report:
              precision    recall  f1-score   support

           0       0.93      0.98      0.96     58139
           1       0.71      0.40      0.51      7028

    accuracy                           0.92     

In [13]:
test

,Unnamed: 0,ACCT_AGE,LIMIT,OUTS,ACCT_RESIDUAL_TENURE,LOAN_TENURE,INSTALAMT,SI_FLG,AGE,VINTAGE,...,INCOME_BAND1_EX05,INCOME_BAND1_F,INCOME_BAND1_G,INCOME_BAND1_H,AGREG_GROUP_#Housing Loan,AGREG_GROUP_#Total Auto Loan,AGREG_GROUP_#Total Xpress Credit,PRODUCT_TYPE_EDUCATION LOAN,PRODUCT_TYPE_HOME LOAN,PRODUCT_TYPE_PERSONAL LOAN
0,0,2.694,729200.0,541543.71,3.308,2192,15247.0,1,38.915,18.765,...,False,False,False,False,False,False,True,False,False,True
1,1,5.652,980500.0,426219.82,2.349,2922,15836.0,1,51.436,15.665,...,False,True,False,False,False,True,False,False,False,False
2,2,5.737,980500.0,413595.82,2.265,2922,15836.0,1,51.521,15.750,...,False,True,False,False,False,True,False,False,False,False
3,3,6.479,735500.0,221620.79,1.607,2953,11996.0,1,33.526,14.702,...,False,True,False,False,False,True,False,False,False,False
4,4,6.394,735500.0,231762.79,1.692,2953,11996.0,1,33.441,14.617,...,False,True,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190676,191688,1.561,461500.0,407286.13,4.441,2192,9409.0,0,27.901,11.054,...,False,False,False,False,False,False,True,False,False,True
190677,191689,3.020,495500.0,309105.77,2.982,2192,9165.0,1,39.862,9.866,...,False,False,False,False,False,False,True,False,False,True
190678,191690,3.097,805500.0,545336.58,2.903,2191,16169.0,1,42.127,13.100,...,False,False,True,False,False,False,True,False,False,True
190679,191691,2.336,625500.0,459398.55,3.666,2192,11977.0,1,25.552,9.838,...,False,False,False,False,False,False,True,False,False,True


In [14]:
test_cat = test.copy()
cat_probs = cat_model.predict_proba(test_cat)[:, 1] 


In [ ]:

test_encoded = test_cat.copy()

# Apply same LabelEncoders used during training
for col in categorical_cols:
    if col in test_encoded.columns:
        le = encoders[col]
        test_encoded[col] = test_encoded[col].astype(str)
        test_encoded[col] = le.transform(test_encoded[col])  # reuse trained encoders

test_numeric = convert_to_numeric(test_encoded)


In [16]:
xgb_probs = xgb_model.predict_proba(test_numeric)[:, 1]  # probability of class 1


In [17]:
# Average the probabilities
ensemble_probs = cat_probs * 0.48 + xgb_probs * 0.52

# Convert probabilities to final binary predictions (threshold = 0.5)
ensemble_preds = (ensemble_probs >= 0.5).astype(int)


In [18]:
# Append predictions to the original test DataFrame
test_result = test.copy()
test_result['CatBoost_Prob'] = cat_probs
test_result['XGBoost_Prob'] = xgb_probs
test_result['Ensemble_Prob'] = ensemble_probs
test_result['Ensemble_Pred'] = ensemble_preds

# Preview
print(test_result[['CatBoost_Prob', 'XGBoost_Prob', 'Ensemble_Prob', 'Ensemble_Pred']].head())


   CatBoost_Prob  XGBoost_Prob  Ensemble_Prob  Ensemble_Pred
0       0.137333      0.021323       0.079328              0
1       0.075621      0.019020       0.047320              0
2       0.160301      0.038752       0.099526              0
3       0.158012      0.013248       0.085630              0
4       0.118226      0.014088       0.066157              0


In [20]:
test_result['TARGET'] = ensemble_preds
test_result

,Unnamed: 0,ACCT_AGE,LIMIT,OUTS,ACCT_RESIDUAL_TENURE,LOAN_TENURE,INSTALAMT,SI_FLG,AGE,VINTAGE,...,AGREG_GROUP_#Total Auto Loan,AGREG_GROUP_#Total Xpress Credit,PRODUCT_TYPE_EDUCATION LOAN,PRODUCT_TYPE_HOME LOAN,PRODUCT_TYPE_PERSONAL LOAN,CatBoost_Prob,XGBoost_Prob,Ensemble_Prob,Ensemble_Pred,TARGET
0,0,2.694,729200.0,541543.71,3.308,2192,15247.0,1,38.915,18.765,...,False,True,False,False,True,0.137333,0.021323,0.079328,0,0
1,1,5.652,980500.0,426219.82,2.349,2922,15836.0,1,51.436,15.665,...,True,False,False,False,False,0.075621,0.019020,0.047320,0,0
2,2,5.737,980500.0,413595.82,2.265,2922,15836.0,1,51.521,15.750,...,True,False,False,False,False,0.160301,0.038752,0.099526,0,0
3,3,6.479,735500.0,221620.79,1.607,2953,11996.0,1,33.526,14.702,...,True,False,False,False,False,0.158012,0.013248,0.085630,0,0
4,4,6.394,735500.0,231762.79,1.692,2953,11996.0,1,33.441,14.617,...,True,False,False,False,False,0.118226,0.014088,0.066157,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190676,191688,1.561,461500.0,407286.13,4.441,2192,9409.0,0,27.901,11.054,...,False,True,False,False,True,0.078453,0.014225,0.046339,0,0
190677,191689,3.020,495500.0,309105.77,2.982,2192,9165.0,1,39.862,9.866,...,False,True,False,False,True,0.260258,0.032575,0.146417,0,0
190678,191690,3.097,805500.0,545336.58,2.903,2191,16169.0,1,42.127,13.100,...,False,True,False,False,True,0.076290,0.009199,0.042744,0,0
190679,191691,2.336,625500.0,459398.55,3.666,2192,11977.0,1,25.552,9.838,...,False,True,False,False,True,0.411417,0.140466,0.275941,0,0


In [21]:
print(test_result['TARGET'].value_counts())


TARGET
0    171310
1     19371
Name: count, dtype: int64


In [ ]:

test_result.to_csv("test_predictions_with_target.csv", index=False)
print(test_result.head())


   Unnamed: 0  ACCT_AGE     LIMIT       OUTS  ACCT_RESIDUAL_TENURE  \
0           0     2.694  729200.0  541543.71                 3.308   
1           1     5.652  980500.0  426219.82                 2.349   
2           2     5.737  980500.0  413595.82                 2.265   
3           3     6.479  735500.0  221620.79                 1.607   
4           4     6.394  735500.0  231762.79                 1.692   

   LOAN_TENURE  INSTALAMT  SI_FLG     AGE  VINTAGE  ...  \
0         2192    15247.0       1  38.915   18.765  ...   
1         2922    15836.0       1  51.436   15.665  ...   
2         2922    15836.0       1  51.521   15.750  ...   
3         2953    11996.0       1  33.526   14.702  ...   
4         2953    11996.0       1  33.441   14.617  ...   

   AGREG_GROUP_#Total Auto Loan  AGREG_GROUP_#Total Xpress Credit  \
0                         False                              True   
1                          True                             False   
2                

In [25]:
import zipfile

In [27]:
zip_filename = "test_predictions_with_target.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write("test_predictions_with_target.csv")

print(f"ZIP file '{zip_filename}' created successfully.")

ZIP file 'test_predictions_with_target.zip' created successfully.
